In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam, SGD, AdamW
import torchvision
import torchvision.datasets as dsets
import numpy as np
import time
import random
from tqdm.auto import tqdm

from torch.optim.lr_scheduler import ReduceLROnPlateau

import pandas as pd
import os
from torch.utils.data import DataLoader, sampler


In [2]:
class CFG:
    # ============== comp exp name =============
    comp_name = 'cifar10'

    # comp_dir_path = './'
    comp_dir_path = '/kaggle/input/'
    comp_folder_name = 'cifar10'
    # comp_dataset_path = f'{comp_dir_path}datasets/{comp_folder_name}/'
    comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'
    exp_name = 'exp_01'

    # ============== model cfg =============
    model_name = 'resnet34'
    in_chans = 16 
    # ============== training cfg =============
    trian_size = 45000
    
    size = 32
    batch_size = 128
    epochs = 50
    lr = 0.01

    # ============== set dataset path =============
    print('set dataset path')

    outputs_path = f'/kaggle/working/outputs/{comp_name}/{exp_name}/'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path + \
        f'{comp_name}-models/'

    log_dir = outputs_path + 'logs/'
    log_path = log_dir + f'{exp_name}.txt'

    num_workers = 4

    seed = 2

set dataset path


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
for dir in [CFG.model_dir, CFG.submission_dir, CFG.log_dir]:
    os.makedirs(dir, exist_ok=True)

In [5]:
def set_seed(seed=None, cudnn_deterministic=True):
    if seed is None:
        seed = 2

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = cudnn_deterministic

def init_logger(log_file):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger
set_seed()
Logger = init_logger(CFG.log_path)

In [6]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [7]:
model = ResNet(ResidualBlock, [3, 4, 6, 3])
model.to(device)

optimizer = AdamW(model.parameters(), lr=CFG.lr)
scheduler = ReduceLROnPlateau(optimizer,'min',verbose=True,patience=4,factor=0.1)

criterion = nn.CrossEntropyLoss()

In [8]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])
mean, std = [0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]
composed_train = transforms.Compose([transforms.RandomRotation(20), # Randomly rotate some images by 20 degrees
                                     transforms.RandomHorizontalFlip(0.1), # Randomly horizontal flip the images
                                     transforms.ColorJitter(brightness = 0.1, # Randomly adjust color jitter of the images
                                                            contrast = 0.1, 
                                                            saturation = 0.1), 
                                     transforms.RandomAdjustSharpness(sharpness_factor = 2,
                                                                      p = 0.1), # Randomly adjust sharpness
                                     transforms.ToTensor(),   # Converting image to tensor
                                     transforms.Normalize(mean, std), # Normalizing with standard mean and standard deviation
                                     transforms.RandomErasing(p=0.75,scale=(0.02, 0.1),value=1.0, inplace=False)])


composed_test = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean, std)])



train_dataset =  dsets.CIFAR10(root='./data', 
                               train=True, 
                               download=True, 
                               transform = composed_train)
valid_dataset =  dsets.CIFAR10(root='./data', 
                               train=True, 
                               download=True, 
                               transform = composed_test)

train_loader = DataLoader(train_dataset,
                          batch_size=CFG.batch_size,
                          sampler=sampler.SubsetRandomSampler(range(CFG.trian_size)),
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                          )

valid_loader = DataLoader(valid_dataset,
                          batch_size=CFG.batch_size,
                          sampler=sampler.SubsetRandomSampler(range(CFG.trian_size, 50000)),
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:
def train_fn(train_loader, model, criterion, optimizer, device):
    model.train()
    losses = AverageMeter()

    for step, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return losses.avg

def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    losses = AverageMeter()
    correct = 0

    for step, (images, labels) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        
        _, yhat = torch.max(y_preds.data, 1)

        correct += (yhat==labels).sum().item()
        
    accuracy = correct/(50000-CFG.trian_size)
    
    
    return losses.avg, accuracy

In [11]:
best_score = -1
best_loss = np.inf
for epoch in range(CFG.epochs):

    start_time = time.time()

        # train
    avg_loss = train_fn(train_loader, model, criterion, optimizer, device)

    # eval
    avg_val_loss, score = valid_fn(valid_loader, model, criterion, device)

    scheduler.step(avg_val_loss)
    

    elapsed = time.time() - start_time

    Logger.info(
        f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
    # Logger.info(f'Epoch {epoch+1} - avgScore: {avg_score:.4f}')
    Logger.info(
        f'Epoch {epoch+1} - avgScore: {score:.4f}')
    
    update_best = score > best_score

    if update_best:
        best_loss = avg_val_loss
        best_score = score

        Logger.info(
            f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
        Logger.info(
            f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')

        torch.save({'model': model.state_dict(),},
                    CFG.model_dir + f'{CFG.model_name}_best.pth')

  0%|          | 0/351 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1 - avg_train_loss: 1.8069  avg_val_loss: 1.6321  time: 47s
Epoch 1 - avgScore: 0.3896
Epoch 1 - Save Best Score: 0.3896 Model
Epoch 1 - Save Best Loss: 1.6321 Model


  0%|          | 0/351 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 49 - avg_train_loss: 0.1950  avg_val_loss: 0.3110  time: 41s
Epoch 49 - avgScore: 0.9026
Epoch 49 - Save Best Score: 0.9026 Model
Epoch 49 - Save Best Loss: 0.3110 Model


  0%|          | 0/351 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 50 - avg_train_loss: 0.1942  avg_val_loss: 0.3106  time: 40s
Epoch 50 - avgScore: 0.9020


In [12]:
test_dataset =  dsets.CIFAR10(root='./data', 
                               train=False, 
                               download=True, 
                               transform = composed_test)

test_loader = DataLoader(test_dataset,
                          batch_size=CFG.batch_size,
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

Files already downloaded and verified


In [13]:
def test_fn(test_loader, model, criterion, device):
    model.eval()
    losses = AverageMeter()
    correct = 0

    for step, (images, labels) in tqdm(enumerate(test_loader), total=len(test_loader)):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        
        _, yhat = torch.max(y_preds.data, 1)

        correct += (yhat==labels).sum().item()

    accuracy = correct/len(test_dataset)

    
    return losses.avg, accuracy

In [14]:
avg_val_loss, score = test_fn(test_loader, model, criterion, device)
print(score)

  0%|          | 0/79 [00:00<?, ?it/s]

0.8948
